In [ ]:
#Only run to mount google drive
#from google.colab import drive
#drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#####https://www.analyticsvidhya.com/blog/2024/10/langgraph-react-function-calling/
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) #logging.INFO
logging.debug("test")
logging.info('test2')

DEBUG:root:test
INFO:root:junk


In [ ]:
!pip install duckduckgo-search
!pip install langchain-community
!pip install langchain-experimental
!pip install langchain-openai
!pip install langchainhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.8 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI
#0. Setting the OpenAI API key from google colab
#from google.colab import userdata
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
#1. GET your OpenAI API key from the OpenAI dashboard
api_key = os.getenv("OPENAI_API_KEY")
#2. Create a new OpenAI instance
#client = OpenAI(api_key=api_key)
#3. Set the chat model
chat_model="gpt-4o-mini"

In [2]:
from langchain import hub
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_openai import ChatOpenAI

# Set up the LLM
llm = ChatOpenAI(
    openai_api_key = api_key,
    temperature = 0,
    model = chat_model)

# Content of the prompt template
template = '''
Answer the following question as best you can.
Do not use a tool if not required.
Question: {question}
'''

# Create the prompt template
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')

# Set up the Python REPL tool
python_repl = PythonAstREPLTool()
python_repl_tool = Tool(
    name = 'Python REPL',
    func = python_repl.run,
    description = '''
    A Python shell. Use this to execute python commands.
    Input should be a valid python command.
    When using this tool, sometimes output is abbreviated - make sure
    it does not look abbreviated before using it in your answer.
    '''
)

# Set up the DuckDuckGo Search tool
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func = search.run,
    description = '''
    A wrapper around DuckDuckGo Search.
    Useful for when you need to answer questions about current events.
    Input should be a search query.
    '''
)

# Create an array that contains all the tools used by the agent
tools = [python_repl_tool, duckduckgo_tool]

# Create a ReAct agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools = tools,
    verbose = True, # explain all reasoning steps
    handle_parsing_errors=True, # continue on error
    max_iterations = 10 # try up to 10 times to find the best answer
)

# Ask your question (replace this with your question)
question = "What is '(2 * 2 / 2) ^ 3'?"
output = agent_executor.invoke({'input': prompt_template.format(question=question)})

/Users/apple/learning/code/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
To solve the expression '(2 * 2 / 2) ^ 3', I will first evaluate the expression inside the parentheses and then raise the result to the power of 3.

Action: Python REPL  
Action Input: (2 * 2 / 2) ** 3  8.0I now know the final answer.  
Final Answer: 8.0

> Finished chain.
